# WeatherPy
----

### Analysis
* As expected, the weather becomes significantly warmer as one approaches the equator (0 Deg. Latitude). More interestingly, however, is the fact that the southern hemisphere tends to be warmer this time of year than the northern hemisphere. This may be due to the tilt of the earth.
* There is no strong relationship between latitude and cloudiness. However, it is interesting to see that a strong band of cities sits at 0, 80, and 100% cloudiness.
* There is no strong relationship between latitude and wind speed. However, in northern hemispheres there is a flurry of cities with over 20 mph of wind.

---

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [5]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
# import time
import datetime
now = datetime.datetime.now()
todaysdate = now.strftime("%m-%d-%Y")


# Dependencies
import openweathermapy.core as owm

# Import API key
from api_keys import api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [6]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)
print(lats)
# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

[ 32.7396638  -69.17942547  12.92797374 ...  67.67002289 -56.51372338
  17.42889274]


627

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [8]:
##setup the lists i will use to store the values for each city into a list that i will put into a dataframe for charting 
lat = []
lng = []
nl = "\n"
City = []
Cloudiness = []
Country = []
Date = []
Humidity = []
Lat = []
Lng = []
Max_Temp = []
Wind_Speed = []  
##Created a append method to add all the values of intrest for the city to the appropriate list
def toappend(ct,cd,co,dt,hu,la,ln,mt,ws):
    City.append(ct)
    Cloudiness.append(cd)
    Country.append(co)
    Date.append(dt)
    Humidity.append(hu)
    Lat.append(la)
    Lng.append(ln)
    Max_Temp.append(mt)
    Wind_Speed.append(ws)

#Method to parse the value of intrest    
def parseinformation(response):
    #using Try and except to catch all the bad Valid [200] responses
    try:
        city_name = response["name"]
        cloudiness = response["clouds"]["all"]
        country = response["sys"]["country"]
        date = response["dt"]
        humidity = response["main"]["humidity"]
        lat = response["coord"]["lat"]
        lng = response["coord"]["lon"]
        max_temp = response["main"]["temp"]
        wind_speed = response["wind"]["speed"]
        appendvalues = [city_name,cloudiness,country,date,humidity,lat,lng,max_temp,wind_speed]
        toappend(*appendvalues)
        print(f"Processing {city_name}: {nl} cloudiness: {cloudiness},{nl} country: {country},date: {date}, humidity {humidity}, lat: {lat}, lng: {lng},wind speed: {wind_speed}, max temp {max_temp}  ")
    except KeyError:
        skipped
        return "Not Found"
        print("City Not Found, Skipping..")
        pass
    

#Loop to do the API query to the owm server
counter = 0
print(f"Len of City is is {len(cities)}")
skipped = 0
for city in cities:
        url = "http://api.openweathermap.org/data/2.5/weather?"
        query_url = url + "appid=" + api_key + "&q=" + str(city)
        params = {"appid": api_key, "q": city}
        response = requests.get(query_url, params=params)
        response = response.json()
        status = parseinformation(response)
        print(f"{city} is index {counter} out of {len(cities)} of cities. {len(cities)-counter} cities left to go.")
        if status == "Not Found":
            skipped += 1
            print(f"City {city}: Not Found")
        counter += 1
#         if counter > 501:
#             break


#load all the list data into a dataframe
listtoadd = [City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max_Temp,Wind_Speed]
city_weather_data = pd.DataFrame(listtoadd)
city_weather_data = city_weather_data.T
#rename columns
city_weather_data = city_weather_data.rename(columns={0:"City",
                                                     1:"Cloudiness",
                                                     2:"Country",
                                                     3:"Date",
                                                     4:"Humidity",
                                                     5:"Lat",
                                                     6:"Lng",
                                                     7:"Max Temp",
                                                     8:"Wind Speed",})

print(f"Total Number of Cities: {len(cities)} Numbers of Cities Parsed: {counter-skipped} Skipped Cities: {skipped} ")
# city_weather_data

Len of City is is 627
Processing Fortuna: 
 cloudiness: 0,
 country: ES,date: 1542160800, humidity 93, lat: 38.18, lng: -1.13,wind speed: 2.6, max temp 287.68  
fortuna is index 0 out of 627 of cities. 627 cities left to go.
Processing Cape Town: 
 cloudiness: 40,
 country: ZA,date: 1542161400, humidity 88, lat: -33.93, lng: 18.42,wind speed: 3.6, max temp 290.15  
cape town is index 1 out of 627 of cities. 626 cities left to go.
Processing Koutiala: 
 cloudiness: 8,
 country: ML,date: 1542162532, humidity 68, lat: 12.39, lng: -5.46,wind speed: 1.16, max temp 294.562  
koutiala is index 2 out of 627 of cities. 625 cities left to go.
Processing Mehamn: 
 cloudiness: 92,
 country: NO,date: 1542162532, humidity 93, lat: 71.03, lng: 27.85,wind speed: 3.66, max temp 274.062  
mehamn is index 3 out of 627 of cities. 624 cities left to go.
Processing Bani Walid: 
 cloudiness: 80,
 country: LY,date: 1542162533, humidity 87, lat: 31.76, lng: 14,wind speed: 2.21, max temp 286.812  
bani walid is

Processing Ust-Koksa: 
 cloudiness: 8,
 country: RU,date: 1542162564, humidity 67, lat: 50.27, lng: 85.62,wind speed: 0.91, max temp 250.737  
ust-koksa is index 43 out of 627 of cities. 584 cities left to go.
Processing Yellowknife: 
 cloudiness: 90,
 country: CA,date: 1542157200, humidity 78, lat: 62.45, lng: -114.38,wind speed: 8.2, max temp 262.15  
yellowknife is index 44 out of 627 of cities. 583 cities left to go.
Processing Agirish: 
 cloudiness: 80,
 country: RU,date: 1542162564, humidity 73, lat: 61.92, lng: 63.02,wind speed: 3.56, max temp 269.587  
agirish is index 45 out of 627 of cities. 582 cities left to go.
belushya guba is index 46 out of 627 of cities. 581 cities left to go.
City belushya guba: Not Found
Processing San Cristobal: 
 cloudiness: 75,
 country: EC,date: 1542160800, humidity 93, lat: -0.39, lng: -78.55,wind speed: 2.1, max temp 286.15  
san cristobal is index 47 out of 627 of cities. 580 cities left to go.
Processing Biak: 
 cloudiness: 76,
 country: ID,d

Processing Barrow: 
 cloudiness: 92,
 country: AR,date: 1542162436, humidity 100, lat: -38.31, lng: -60.23,wind speed: 4.76, max temp 286.887  
barrow is index 85 out of 627 of cities. 542 cities left to go.
Processing Colquechaca: 
 cloudiness: 20,
 country: BO,date: 1542162597, humidity 40, lat: -18.7, lng: -66,wind speed: 0.26, max temp 279.637  
colquechaca is index 86 out of 627 of cities. 541 cities left to go.
Processing Pevek: 
 cloudiness: 0,
 country: RU,date: 1542163023, humidity 79, lat: 69.7, lng: 170.27,wind speed: 3.01, max temp 255.037  
pevek is index 87 out of 627 of cities. 540 cities left to go.
Processing Blyth: 
 cloudiness: 92,
 country: GB,date: 1542160200, humidity 81, lat: 55.12, lng: -1.5,wind speed: 5.7, max temp 282.8  
blyth is index 88 out of 627 of cities. 539 cities left to go.
Processing Kaspiysk: 
 cloudiness: 90,
 country: RU,date: 1542159000, humidity 76, lat: 42.88, lng: 47.64,wind speed: 7, max temp 282.15  
kaspiysk is index 89 out of 627 of citi

Processing Busselton: 
 cloudiness: 0,
 country: AU,date: 1542162759, humidity 89, lat: -33.64, lng: 115.35,wind speed: 4.36, max temp 292.712  
busselton is index 126 out of 627 of cities. 501 cities left to go.
Processing Hilo: 
 cloudiness: 1,
 country: US,date: 1542160560, humidity 62, lat: 19.71, lng: -155.08,wind speed: 3.1, max temp 295.7  
hilo is index 127 out of 627 of cities. 500 cities left to go.
Processing Mount Isa: 
 cloudiness: 20,
 country: AU,date: 1542160800, humidity 32, lat: -20.73, lng: 139.49,wind speed: 3.1, max temp 308.15  
mount isa is index 128 out of 627 of cities. 499 cities left to go.
Processing Flinders: 
 cloudiness: 75,
 country: AU,date: 1542157200, humidity 77, lat: -34.58, lng: 150.85,wind speed: 3.6, max temp 294.15  
flinders is index 129 out of 627 of cities. 498 cities left to go.
Processing Avarua: 
 cloudiness: 75,
 country: CK,date: 1542160800, humidity 78, lat: -21.21, lng: -159.78,wind speed: 3.6, max temp 301.15  
avarua is index 130 out

Processing Sukhobuzimskoye: 
 cloudiness: 75,
 country: RU,date: 1542159000, humidity 72, lat: 56.5, lng: 93.28,wind speed: 5, max temp 263.15  
sukhobuzimskoye is index 168 out of 627 of cities. 459 cities left to go.
Processing Rio Gallegos: 
 cloudiness: 0,
 country: AR,date: 1542160800, humidity 30, lat: -51.62, lng: -69.22,wind speed: 6.2, max temp 288.15  
rio gallegos is index 169 out of 627 of cities. 458 cities left to go.
Processing Bambous Virieux: 
 cloudiness: 75,
 country: MU,date: 1542160800, humidity 88, lat: -20.34, lng: 57.76,wind speed: 3.1, max temp 297.15  
bambous virieux is index 170 out of 627 of cities. 457 cities left to go.
Processing Shelburne: 
 cloudiness: 40,
 country: CA,date: 1542160200, humidity 68, lat: 44.08, lng: -80.2,wind speed: 7.7, max temp 270.08  
shelburne is index 171 out of 627 of cities. 456 cities left to go.
Processing Banjarmasin: 
 cloudiness: 40,
 country: ID,date: 1542157200, humidity 94, lat: -3.32, lng: 114.59,wind speed: 1.5, max 

...

Processing Amarpur: 
 cloudiness: 0,
 country: IN,date: 1542157200, humidity 93, lat: 23.52, lng: 91.65,wind speed: 1.51, max temp 291.15  
amarpur is index 427 out of 627 of cities. 200 cities left to go.
Processing Pangnirtung: 
 cloudiness: 12,
 country: CA,date: 1542163165, humidity 100, lat: 66.15, lng: -65.72,wind speed: 1.96, max temp 255.737  
pangnirtung is index 428 out of 627 of cities. 199 cities left to go.
Processing Thinadhoo: 
 cloudiness: 92,
 country: MV,date: 1542162848, humidity 100, lat: 0.53, lng: 72.93,wind speed: 8.06, max temp 301.337  
thinadhoo is index 429 out of 627 of cities. 198 cities left to go.
Processing Christchurch: 
 cloudiness: 0,
 country: NZ,date: 1542160800, humidity 72, lat: -43.53, lng: 172.64,wind speed: 7.7, max temp 291.15  
christchurch is index 430 out of 627 of cities. 197 cities left to go.
Processing Tual: 
 cloudiness: 32,
 country: ID,date: 1542162849, humidity 100, lat: -5.67, lng: 132.75,wind speed: 2.46, max temp 302.812  
tual i

Processing Coihaique: 
 cloudiness: 90,
 country: CL,date: 1542160800, humidity 57, lat: -45.58, lng: -72.07,wind speed: 4.1, max temp 283.04  
coihaique is index 469 out of 627 of cities. 158 cities left to go.
Processing Price: 
 cloudiness: 1,
 country: US,date: 1542160380, humidity 40, lat: 39.6, lng: -110.81,wind speed: 3.6, max temp 271.45  
price is index 470 out of 627 of cities. 157 cities left to go.
Processing Bon Air: 
 cloudiness: 90,
 country: US,date: 1542161700, humidity 88, lat: 37.52, lng: -77.56,wind speed: 2.36, max temp 281.31  
bon air is index 471 out of 627 of cities. 156 cities left to go.
Processing Kidal: 
 cloudiness: 64,
 country: ML,date: 1542162877, humidity 33, lat: 18.44, lng: 1.41,wind speed: 1.31, max temp 291.237  
kidal is index 472 out of 627 of cities. 155 cities left to go.
Processing Vila Franca do Campo: 
 cloudiness: 75,
 country: PT,date: 1542160800, humidity 93, lat: 37.72, lng: -25.43,wind speed: 8.7, max temp 290.15  
vila franca do campo 

Processing Highland Springs: 
 cloudiness: 90,
 country: US,date: 1542161700, humidity 76, lat: 37.55, lng: -77.33,wind speed: 3.1, max temp 281.39  
highland springs is index 510 out of 627 of cities. 117 cities left to go.
Processing Yerbogachen: 
 cloudiness: 48,
 country: RU,date: 1542162908, humidity 68, lat: 61.28, lng: 108.01,wind speed: 2.56, max temp 254.487  
yerbogachen is index 511 out of 627 of cities. 116 cities left to go.
Processing La Tuque: 
 cloudiness: 56,
 country: CA,date: 1542162908, humidity 67, lat: 47.44, lng: -72.79,wind speed: 4.66, max temp 264.887  
la tuque is index 512 out of 627 of cities. 115 cities left to go.
Processing Iquique: 
 cloudiness: 75,
 country: CL,date: 1542160800, humidity 77, lat: -20.22, lng: -70.14,wind speed: 2.1, max temp 291.15  
iquique is index 513 out of 627 of cities. 114 cities left to go.
Processing Kapit: 
 cloudiness: 92,
 country: MY,date: 1542162908, humidity 92, lat: 2.02, lng: 112.94,wind speed: 1.31, max temp 298.062  

Processing Kiunga: 
 cloudiness: 48,
 country: PG,date: 1542163243, humidity 42, lat: -6.12, lng: 141.3,wind speed: 1.16, max temp 308.212  
kiunga is index 552 out of 627 of cities. 75 cities left to go.
Processing Penzance: 
 cloudiness: 75,
 country: GB,date: 1542160200, humidity 93, lat: 50.12, lng: -5.53,wind speed: 7.7, max temp 285.15  
penzance is index 553 out of 627 of cities. 74 cities left to go.
Processing Dusti: 
 cloudiness: 92,
 country: TJ,date: 1542162936, humidity 100, lat: 37.35, lng: 68.68,wind speed: 2.71, max temp 276.187  
dusti is index 554 out of 627 of cities. 73 cities left to go.
Processing Katherine: 
 cloudiness: 75,
 country: AU,date: 1542160800, humidity 52, lat: -14.47, lng: 132.26,wind speed: 3.6, max temp 306.15  
katherine is index 555 out of 627 of cities. 72 cities left to go.
Processing Nakapiripirit: 
 cloudiness: 0,
 country: UG,date: 1542162936, humidity 86, lat: 1.81, lng: 34.71,wind speed: 1.56, max temp 287.862  
nakapiripirit is index 556 

Processing Barcelos: 
 cloudiness: 0,
 country: PT,date: 1542160800, humidity 81, lat: 41.53, lng: -8.62,wind speed: 1.5, max temp 282.51  
barcelos is index 595 out of 627 of cities. 32 cities left to go.
Processing Cam Ranh: 
 cloudiness: 0,
 country: VN,date: 1542162964, humidity 89, lat: 11.92, lng: 109.15,wind speed: 3.21, max temp 299.837  
cam ranh is index 596 out of 627 of cities. 31 cities left to go.
Processing Mezen: 
 cloudiness: 92,
 country: RU,date: 1542162964, humidity 89, lat: 65.85, lng: 44.24,wind speed: 2.31, max temp 273.437  
mezen is index 597 out of 627 of cities. 30 cities left to go.
karkaralinsk is index 598 out of 627 of cities. 29 cities left to go.
City karkaralinsk: Not Found
Processing Jieshi: 
 cloudiness: 0,
 country: CN,date: 1542162965, humidity 89, lat: 22.82, lng: 115.83,wind speed: 8.71, max temp 296.812  
jieshi is index 599 out of 627 of cities. 28 cities left to go.
Processing Statesboro: 
 cloudiness: 90,
 country: US,date: 1542160740, humidi

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [ ]:
#map the latitude, temp, humidity, cloudiness and wind to variables to use for charting.
latitude = city_weather_data["Lat"]
humidity = city_weather_data["Humidity"]
cloudiness = city_weather_data["Cloudiness"]

#converted K -> F Temp
temp = city_weather_data["Max Temp"] - 273.15
temp = temp * (9/5) + 32
temp = temp

#converted wind from meter/sec to mhp
wind = city_weather_data["Wind Speed"] * 2.23694


### Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

#### Latitude vs. Temperature Plot

In [ ]:

#Scatter plot with Lat and temp with edgedcolors
plt.scatter(latitude,temp,edgecolors="black")

#adding lables to plot
plt.xlabel("Latitude")
plt.ylabel("Max Temp(F)")
plt.title(f"City Latitude vs. Max Temp ({todaysdate})")

#using grid and saving graph
plt.grid()
plt.savefig(f"City Latitude vs. Max Temp({todaysdate}).jpg")

#### Latitude vs. Humidity Plot

In [ ]:
#Scatter plot with Lat and humidity with edgedcolors
plt.scatter(latitude,humidity,edgecolors="black")

#adding lables to plot
plt.xlabel("Latitude")
plt.ylabel("Humidity(%)")
plt.title(f"City Latitude vs. Humidity({todaysdate})")
plt.grid()

#looked like there was a bad data set in the pull, went over 100%
plt.ylim(0,105)

#using grid and saving graph
plt.savefig(f"City Latitude vs. Humidity({todaysdate}).jpg")


#### Latitude vs. Cloudiness Plot

In [ ]:
#Scatter plot with Lat and temp with edgedcolors
plt.scatter(latitude,cloudiness,edgecolors="black")

#adding lables to plot
plt.xlabel("Latitude")
plt.ylabel("Cloudiness(%)")
plt.title(f"City Latitude vs. Cloudiness({todaysdate})")

#using grid and saving graph
plt.grid()
plt.savefig(f"City Latitude vs. Cloudiness({todaysdate}).jpg")

#### Latitude vs. Wind Speed Plot

In [ ]:
#Scatter plot with Lat and temp with edgedcolors
plt.scatter(latitude,wind,edgecolors="black")

#adding lables to plot
plt.xlabel("Latitude")
plt.ylabel("Wind Speed(mph)")
plt.title(f"City Latitude vs. Max Temp({todaysdate})")

#using grid and saving graph
plt.grid()
plt.savefig(f"City Latitude vs. Max Temp({todaysdate}).jpg")